# Lifting State Up

Several components may need to "react" to changing data of other components. React recommends "lifting" the shared state up to the closest common ancestor. Let's start with an example:

In [ ]:
function BoilingVerdict(props) {
  if (props.celsius >= 100) {
    return <p>The water would boil.</p>;
  }
  return <p>The water would not boil.</p>;
}

Next we will create another component `Calculator` that uses this component:

In [ ]:
class Calculator extends React.Component {
  constructor(props) {
    super(props);
    this.handleChange = this.handleChange.bind(this);
    this.state = {temperature: ''};
  }

  handleChange(e) {
    this.setState({temperature: e.target.value});
  }

  render() {
    const temperature = this.state.temperature;
    return (
      <fieldset>
        <legend>Enter temperature in Celsius:</legend>
        <input
          value={temperature}
          onChange={this.handleChange} />
        <BoilingVerdict
          celsius={parseFloat(temperature)} />
      </fieldset>
    );
  }
}

Can you explain what happens here and how this code works?

Let's try to add a second input now to the `Calculator`. Let's get a `TemperatureInput` to use that will have a `scale` prop that can be either "c" or "f":

In [ ]:
const scaleNames = {
  c: 'Celsius',
  f: 'Fahrenheit'
};

class TemperatureInput extends React.Component {
  constructor(props) {
    super(props);
    this.handleChange = this.handleChange.bind(this);
    this.state = {temperature: ''};
  }

  handleChange(e) {
    this.setState({temperature: e.target.value});
  }

  render() {
    const temperature = this.state.temperature;
    const scale = this.props.scale;
    return (
      <fieldset>
        <legend>Enter temperature in {scaleNames[scale]}:</legend>
        <input value={temperature}
               onChange={this.handleChange} />
      </fieldset>
    );
  }
}

Let's use this to render two separate temperature inputs:

In [ ]:
class Calculator extends React.Component {
  render() {
    return (
      <div>
        <TemperatureInput scale="c" />
        <TemperatureInput scale="f" />
      </div>
    );
  }
}

Make sure you take your time here and understand everything, as this may look weird at first.

Now we have two inputs, but when we enter the temperature in one of them, the other doesn't update.

And we also CANNOT render `BoilingVerdict` IN `Calculator`. Why can't we do this? (Make sure you understand why, as it is an important concept.)

The answer is because `Calculator` doesn't know the current temperature because it is hidden inside `TemperatureInput`. So we need to ***PASS THE STATE UP***.

First, we need to create two functions that will convert farenheit to celcius, and vice versa. Make sure the functions return the values.

In [ ]:
function toCelsius(fahrenheit) {
  return (fahrenheit - 32) * 5 / 9;
}

function toFahrenheit(celsius) {
  return (celsius * 9 / 5) + 32;
}

We will use these two functions in one function: `tryConvert(temperature, convert)`, where `temperature` is the input value (which is a string. This means the input could be letters), `convert` is the function that we will call to convert `temperature`. If `temperature` is not a valid value, then the function should return an empty string. Otherwise, it should return the valid string value after conversion. Make this function.

Here are some hints:

`parseFloat(temp)`  
`Number.isNaN(val)` will check if `val` is null  
Use `someNumber.toString()` to convert any `Number` value to string  
`Math.round(val)`
`convert(input)` where `convert` is either `toCelsius` or `toFarenheit`

In [ ]:
function tryConvert(temperature, convert){
    const input = parseFloat(temperature);
    if (Number.isNaN(input)){
        return '';
    }
    const output = convert(input);
    const rounded = Math.round(output*1000)/1000;
    return rounded.toString();
}

Because we want the temperatures to be in sync with each other, we need to share states. We will remove the local state from `TemperatureInput` and move it to the `Calculator` instead. So `Calculator` can make the two props of the `TemperatureInput` components be in sync.

1. Replace `this.state.temperature` with `this.props.temperature` in `TemperatureInput`. This will look like:

In [ ]:
render() {
    // Before: const temperature = this.state.temperature;
    const temperature = this.props.temperature;
    // ...

Remember that props are read-only. This means that the `TemperatureInput` CANNOT control it anymore by using `this.setState()`.

We can solve this by making a component "controlled" just liked we did using the DOM:   
`<input value="something onChange={this.handleChange}">`  
We will make `TemperatureChange` also accept both `temperature` and `onTemperatureChange` props from `Calculator`.

This gets added to `TemperatureInput` to call `this.props.onTemperatureChange`:

In [ ]:
handleChange(e){
    //Before: this.setState({temperature: e.target.value});
    this.props.onTemperatureChange(e.target.value);
}

We also could have named them `value` and `onChange`, like how it is for `<input>`.

This makes the new `TemperatureInput`:

In [ ]:
class TemperatureInput extends React.Component {
  constructor(props) {
    super(props);
    this.handleChange = this.handleChange.bind(this);
  }

  handleChange(e) {
    this.props.onTemperatureChange(e.target.value);
  }

  render() {
    const temperature = this.props.temperature;
    const scale = this.props.scale;
    return (
      <fieldset>
        <legend>Enter temperature in {scaleNames[scale]}:</legend>
        <input value={temperature}
               onChange={this.handleChange} />
      </fieldset>
    );
  }
}

So here is what is happening:

When its own `TemperatureInput` changes from `<input>`, `handleChange` will occur, which will then run the function that is assigned to it in the `Calculator` component. For example, if the `this.props.scale` that is passed is 'c', then:  
`this.props.onTemperatureChange(e.target.value);`  
is going to be:  
`this.handleCelsiusChange(e.target.value)`  
Which will convert the `this.props.temperature`, to the correct value for the scale. So when changing the celsius one here, it will also change the fahrenheit one, except it uses `this.handleFahrenheitChange`.  
<br>
This also happens when changing the fahrenheit one, but flip everything of course.

So here is the code for the `Calculator`:

In [ ]:
class Calculator extends React.Component {
  constructor(props) {
    super(props);
    this.handleCelsiusChange = this.handleCelsiusChange.bind(this);
    this.handleFahrenheitChange = this.handleFahrenheitChange.bind(this);
    this.state = {temperature: '', scale: 'c'};
  }

  handleCelsiusChange(temperature) {
    this.setState({scale: 'c', temperature});
  }

  handleFahrenheitChange(temperature) {
    this.setState({scale: 'f', temperature});
  }

  render() {
    const scale = this.state.scale;
    const temperature = this.state.temperature;
    const celsius = scale === 'f' ? tryConvert(temperature, toCelsius) : temperature;
    const fahrenheit = scale === 'c' ? tryConvert(temperature, toFahrenheit) : temperature;

    return (
      <div>
        <TemperatureInput
          scale="c"
          temperature={celsius}
          onTemperatureChange={this.handleCelsiusChange} />
        <TemperatureInput
          scale="f"
          temperature={fahrenheit}
          onTemperatureChange={this.handleFahrenheitChange} />
        <BoilingVerdict
          celsius={parseFloat(celsius)} />
      </div>
    );
  }
}

1. Let's look at the `render()`
2. `const scale = this.state.scale;` will change to 'c' or 'f' DEPENDING ON WHICH HANDLER GETS CALLED. So if you type on the top `<TemperatureInput>`, then it will run `this.handleCelsiusChange`, and `const scale = 'c'`. If you type on the bottom, it will run `this.handleFahrenheitChange`, and `const scale ='f'`.
3. `const temperature = this.state.temperature;` will be whatever the user typed in the box they are typing in. Because of `this.state.scale`, the `Calculator` also knows which scale the `temperature` is in. This change in value occurs when changed either the top or bottom `<TemperatureInput>`, which runs the handlers.
4. `const celsius = scale === 'f' ? try...;` will first check to see if the current scale that is "selected" from the handler is 'f'. If it is 'f', it will convert it to celsius by using the `tryConvert` function we created. So then this value will be assigned to `const celsius`. If the current scale that is "selected" from the handler is NOT 'f', then that means that there is no need to convert the temperature, so `const celsius = temperature` will run instead.
5. `const fahrenheit = scale === 'c' ....` Same as the last step, but flip everything.
6. For the first `<TemperatureInput>`, note the props that we are passing. We are passing the correct scale, the correct temperature that is ALREADY converted FOR the component, and also the handler that will run when changing the value of this component.
7. The second `<TemperatureInput>` will also act the same as above, but flip.

Try to get the app working!!